# LightGCN

## 1. Set up and imports

In [1]:
import os
import sys

import numpy as np
import pandas as pd
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.timer import Timer
from recommenders.models.deeprec.models.graphrec.lightgcn import LightGCN
from recommenders.models.deeprec.DataModel.ImplicitCF import ImplicitCF
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.evaluation.python_evaluation import ndcg_at_k
from recommenders.utils.constants import SEED as DEFAULT_SEED
from recommenders.models.deeprec.deeprec_utils import prepare_hparams
from recommenders.utils.notebook_utils import store_metadata

print(f"System version: {sys.version}")
print(f"Numpy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"Tensorflow version: {tf.__version__}")

System version: 3.10.14 | packaged by conda-forge | (main, Mar 20 2024, 12:51:49) [Clang 16.0.6 ]
Numpy version: 1.26.4
Pandas version: 2.3.3
Tensorflow version: 2.20.0


### 1.1. Global variables

In [ ]:
# Top 20 items
TOP_K = 20

# Model initial hyperparameters
EMBED_SIZE = 64
N_LAYERS = 3

BATCH_SIZE = 16384
DECAY = 0.0001
EPOCHS = 50
LEARNING_RATE = 0.005
EVAL_EPOCHS = 5

In [3]:
yaml_file = "lightgcn.yaml"

## 2. Data preprocessing

### 2.1. Load and preprocess the data

In [4]:
df = pd.read_csv(
    "../data/train_2_long.csv",
    header = 0,
    names = ["userID", "itemID"]
    )
df.head()

,userID,itemID
0,0,28261
1,0,388
2,0,5731
3,0,401
4,0,28284


In [5]:
# Add rating column
df["rating"] = 1.0
df.head()

,userID,itemID,rating
0,0,28261,1.0
1,0,388,1.0
2,0,5731,1.0
3,0,401,1.0
4,0,28284,1.0


### 2.2. Split data

In [6]:
train, test = python_stratified_split(df, ratio=0.75)

### 2.3. Process data
Using `ImplicitCF` class
* creates an adjacency matrix for user-item graph

In [7]:
# Create data object
data = ImplicitCF(train=train, test=test, seed=DEFAULT_SEED)

## 3. Model

### 3.1. Prepare hyperparameters

In [8]:
hparams = prepare_hparams(yaml_file,
                          n_layers=N_LAYERS,
                          batch_size=BATCH_SIZE,
                          decay=DECAY,
                          epochs=EPOCHS,
                          learning_rate=LEARNING_RATE,
                          eval_epoch=EVAL_EPOCHS,
                          top_k=TOP_K,
                         )

### 3.2. Create and train model

In [ ]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [9]:
model = LightGCN(hparams, data, seed=DEFAULT_SEED)

Already create adjacency matrix.
Already normalize adjacency matrix.
Using xavier initialization.


I0000 00:00:1764389641.709805 19577641 mlir_graph_optimization_pass.cc:437] MLIR V1 optimization pass is not enabled


In [10]:
with Timer() as train_time:
    model.fit()

print(f"Took {train_time.interval} seconds for training.")

Epoch 1 (train)2431.0s: train loss = 0.20171 = (mf)0.20055 + (embed)0.00117


KeyboardInterrupt: 

## 4. Evaluate

### 4.1. Generate recommendations

In [ ]:
top_k_recs = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

top_k_recs.head()

### 4.2. Evaluate using NDCG@20

In [ ]:
eval_ndcg = ndcg_at_k(test, top_k_recs, k=TOP_K)

print(f"NCDG@20:\t{eval_ndcg}")

## 5. Tune model parameters

In [ ]:
import itertools

### 5.1. Start with regularization and learning rate search on fixed architecture.

In [ ]:
def evaluate_model(batch_size, decay, emb_dim, epochs, layers, lr):
    """Evaluate a LightGCN model given hyperparameters."""

    hparams = (yaml_file,
                embed_size=embed_dim,
                n_layers=layers,
                batch_size=batch_size,
                decay=decay,
                epochs=epochs,
                learning_rate=lr,
                eval_epoch=EVAL_EPOCHS,
                top_k=TOP_K)

    model = LightGCN(hparams, data, seed=SEED)
    model.fit()

    recs = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)
    ndcg = ndcg_at_k(test, recs, k=TOP_K)

    return (batch_size, decay, emb_dim, epochs, layers, lr, ndcg)

In [ ]:
# Fixed architecture
BATCH_SIZE = 8192
EMBED_SIZE = 64
N_LAYERS = 2

# Grid search
LR = [5e-4, 1e-3, 2e-3]     # learning rate
L2 = [1e-6, 1e-5, 1e-4]     # decay

# Create all LR and L2 combos
lr_l2_combos = list(itertools.product(LR, L2))

results = []

for (lr, l2) in lr_l2_combos:
    # result = evaluate_model(
    #     batch_size=BATCH_SIZE,
    #     decay=l2,
    #     emb_dim=EMBED_SIZE,
    #     epochs=20,
    #     layers=N_LAYERS,
    #     lr=lr
    # )

    # results.append()
    print(lr, l2)


In [ ]:
best_cur_params = max(results, key=lambda x: x[:-1])
BEST_DECAY = best_cur_params[1]
BEST_LR = best_cur_params[5]
best_cur_ndcg = best_cur_params[-1]

print(f"decay: {BEST_DECAY}, lr: {BEST_LR}, ndcg: {best_cur_ndcg}")

### 5.2. Find best embedding size and layers with fixed regularization and decay.

In [ ]:
EMBED_SIZES = [32, 64]
LAYERS = [1, 2]

emb_layers_combos = list(itertools.product(EMBED_SIZES, LAYERS))



## 6. Recommend

### 6.1. Train model with best params and on full dataset

In [ ]:
# import tensorflow as tf
# from tensorflow.python.framework import test_util as tf_test
# print(tf.config.list_physical_devices('GPU'))
# print(tf_test.gpu_device_name())

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
/device:GPU:0


2025-11-28 21:04:56.832146: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-11-28 21:04:56.832166: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


### 6.2 Generate predictions file